In [1]:
import numpy as np
from functools import partial
import sobol_seq

**Problem 1 Pak Kun**

In [2]:
def objective_function(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    return np.array([f1,f2])

dim = 2
boundaries = np.array([(-10,10) for _ in range (dim)])
pop_size=100
max_gen=1000
F_init=0.5
CR_init=0.5
num_l=20
theta=1e-3
tau_d=0.4
s_max=100
print_gen=True
Hm = 50

seed = np.random.randint(0,100)
seed

8

# Objective Function

In [3]:
def root_objective_function(x:np.ndarray):
    res = 0
    F_array = objective_function(x)
    for f in F_array:
        res +=(f)**2
    return res

# def root_objective_function(x:np.ndarray):
#     res = 0
#     F_array = objective_function(x)
#     for f in F_array:
#         res +=abs(f)
#     return res

# def root_objective_function(x:np.ndarray):
#     F_array = objective_function(x)
#     denom = 0
#     for f in F_array:
#         denom += np.abs(f)
#     res = 1/(1+denom)
#     return -res

# RAGA Algorithm

In [4]:
"""Characteristic Function"""
def chi_p(delta_j, rho):
    return 1 if delta_j <= rho else 0

"""Repulsion Function"""
def repulsion_function(x,
                       archive,
                       objective_func=root_objective_function,
                       beta=30,
                       rho=1e-8):
    f_x = objective_func(x)
    Rx = 0
    for x_star in archive:
        delta_j = np.linalg.norm(x-x_star)
        Rx += np.exp(-delta_j) * chi_p(delta_j, rho)
    Rx *= beta
    Rx += f_x
    return Rx

"""Fitness Function"""
def fitness_function(x,
                     archive,
                     objective_func=root_objective_function,
                     repulsion_func=repulsion_function):
    f_x = objective_func(x)
    if archive == []:
        return f_x
    else:
        return repulsion_func(x,archive)
    
# # Test the Functiuons
# arch = [np.array([1,2]),np.array([3,4]),np.array([-6.43,0.15])]
# x = np.array([-6.437160,0.155348])
# print(fitness_function(x,
#                        arch,
#                        objective_func=root_objective_function,
#                        repulsion_func=partial(repulsion_function,
#                                               objective_func = root_objective_function,
#                                               beta=1000,rho=0.01)))

In [5]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim: int,
                    npoint:int,
                    low=-10,
                    high=10,
                    sobol = True):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    if sobol == True:
        # Generate Sobol sequence points
        sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)
        # Scale the Sobol points to fit within the specified boundaries
        scaled_points = []
        for i in range(dim):
            a, b = boundaries[i]
            scaled_dim = a + sobol_points[:, i] * (b - a)
            scaled_points.append(scaled_dim)
        # Transpose the scaled points to get points per dimension
        scaled_points = np.array(list(map(list, zip(*scaled_points))))
    
    else:
        scaled_points = np.zeros((npoint, dim))
        for i in range(dim):
            min_val, max_val = boundaries[i]
            scaled_points[:, i] = np.random.uniform(min_val, max_val, npoint)

    return scaled_points

# def initialize_population(pop_size, dimensions, bounds):
#     population = np.random.rand(pop_size, dimensions)
#     lower_bounds, upper_bounds = np.asarray(bounds).T
#     diff = np.fabs(lower_bounds - upper_bounds)
#     return lower_bounds + population * diff

In [6]:
"""Roullete wheel selection"""
def selection(population: np.ndarray,
              fitness: np.ndarray):
    population_size = population.shape[0]
    selection_probs = np.array([1 / (fit + 1) for fit in fitness]) # add one to avoid negative probability
    total_probs = sum(selection_probs)
    selection_probs = np.array([prob / total_probs for prob in selection_probs])
    selected_indices = np.random.choice(a=np.arange(population_size),size=population_size,p=selection_probs)
    selected_population = np.array([population[i] for i in selected_indices])
    return selected_population

"""Single point crossover"""
def crossover(parent1, parent2):
    dimension = len(parent1)
    crossover_point = np.random.randint(1, dimension)
    offspring1 = np.append(parent1[:crossover_point], parent2[crossover_point:])
    offspring2 = np.append(parent2[:crossover_point], parent1[crossover_point:])
    return offspring1, offspring2

"""One point mutation"""
def mutate(individual,mutation_rate, boundaries):
    for j in range(len(individual)):
        if np.random.random() < mutation_rate:
            individual[j] = np.random.uniform(boundaries[j][0], boundaries[j][1])
    return individual

def recombination(population: np.ndarray,mutation_rate, boundaries):
    offspring_population = []
    population_size = population.shape[0]
    for i in range(0, population_size, 2):
        parent1 = population[i]
        parent2 = population[i + 1]
        offspring1, offspring2 = crossover(parent1=parent1,parent2=parent2)
        offspring1 = mutate(individual=offspring1,mutation_rate=mutation_rate, boundaries=boundaries)
        offspring2 = mutate(individual=offspring2,mutation_rate=mutation_rate, boundaries=boundaries)
        offspring_population.extend([offspring1, offspring2])
    offspring_population = np.array(offspring_population)
    return offspring_population

In [7]:
"""FInd the closest points in a set to an initial point"""
def closest_solution(initial_point,set_of_points):
    diff = set_of_points-initial_point
    distances = np.linalg.norm(diff,axis=1)
    id_min_dist = np.argmin(distances)
    return id_min_dist,set_of_points[id_min_dist]

# # test the function
# alpha = np.array([1, 1])
# B = np.array([[5, 2],
#               [3, 4],
#               [5, 5]])
# closest_solution(alpha,B)

In [23]:
"""Calculate Euclidean distances and select t closest individuals"""
def subpopulating(individual, 
                  population, 
                  t,
                  return_index = False,
                  show_distances = False): 
    """Input"""
    # individual
    # population
    # t: max number of units in a subpopulation

    """Algorithm"""
    # Calculate the Euclidean distances from the individual to all others in the population
    distances = np.sqrt(np.sum((population - individual) ** 2, axis=1))
    # Get the indices of the individuals with the smallest distances
    closest_indices = np.argsort(distances)[:t]
    # Form the subpopulation with the closest individuals
    subpop = population[closest_indices]

    if show_distances == True:
        print(f'Distance: \n{distances[:t]}')
    if return_index == True:
        if t == 1:
            return closest_indices,subpop.flatten()
        else:
            return closest_indices,subpop
    else:
        if t == 1:
            return subpop.flatten()
        else:
            return subpop

# # Test the function
# # Assuming P is a numpy array of individuals where each individual is a point in n-dimensional space
# np.random.seed(0)
# P = np.random.rand(10, 2)  # Example: 100 individuals in a 2-dimensional space
# print(f"P:\n{P}")
# print("")

# # The number of individuals to select with the smallest Euclidean distances
# t = 4
# # Forming subpopulations for each individual in P
# subpopulations = [subpopulating(xi, P, t) for xi in P]
# # print(f"subpopulations:{subpopulations}")
# print("")

# # Now subpopulations is a list of numpy arrays, each containing the t closest individuals to each xi in P (including xi itself)
# # For example, to access the subpopulation for the first individual in P:
# subpopulation_first_individual = subpopulations[0]
# print(f"subpopulation_first_individual:\n{subpopulation_first_individual}")

P:
[[0.5488135  0.71518937]
 [0.60276338 0.54488318]
 [0.4236548  0.64589411]
 [0.43758721 0.891773  ]
 [0.96366276 0.38344152]
 [0.79172504 0.52889492]
 [0.56804456 0.92559664]
 [0.07103606 0.0871293 ]
 [0.0202184  0.83261985]
 [0.77815675 0.87001215]]


subpopulation_first_individual:
[[0.5488135  0.71518937]
 [0.4236548  0.64589411]
 [0.60276338 0.54488318]
 [0.43758721 0.891773  ]]


## Algorithm 1 - Gong

In [9]:
def update_archive(x: np.ndarray,
                   objective_function,
                   archive,
                   theta,
                   tau_d,
                   s_max):
    """Input"""
    # x : Individual
    # theta : accuracy level
    # tau_d : distance radius
    # s_max : maximum archive size
    # archive : archive
    # s : archive current size

    f_x = objective_function(x)
    s = len(archive) # archive current size
    if f_x < theta: # x is a root
        # print(f'f({x})= {f_x}')
        if s == 0: # archive is empty
            archive.append(x)
            s+=1
        else:
            """Find the closest solution x_prime ∈ archive to x in the decision space"""
            dist_min = np.linalg.norm(x-archive[0])
            idx_min = 0
            x_prime= archive[idx_min]
            for i in range(1,len(archive)): 
                dist = np.linalg.norm(x-archive[i])
                if dist < dist_min:
                    dist_min = dist
                    x_prime = archive[i]
                    idx_min = i
            f_x_prime = root_objective_function(x_prime)
            if dist_min < tau_d: # x and x_prime are too close
                if f_x < f_x_prime:
                    x_prime = x
                    archive[idx_min] = x_prime
            else:
                if s < s_max:
                    archive.append(x)
                    s += 1
                else:       # archive is full
                    if f_x<f_x_prime:
                        x_prime = x
                        archive[idx_min] = x_prime
    return archive


# # Test the function
# x = np.array([-6.437160,0.155348]) # Individual
# theta = 1e-4 # accuracy level
# tau_d = 1e-1 # distance radius
# s_max = 3 # maximum archive size
# archive = [np.array([0,0]),np.array([1,2]),np.array([-6.4,0])] # archive
# update_archive(x,root_objective_function,archive,theta,tau_d,s_max)

In [10]:
def update_parameter(M_F,
                     M_CR,
                     Hm:int):
    """Input"""
    # MF: Historical memories of scaling factor of DE as F
    # MCR:Historical memories crossover rate of DE as CR
    # Hm: Size of Historical Memories

    # Randomly select an index
    hi = np.random.randint(0, Hm)
    # Generate Fi using the Cauchy distribution with the location parameter MF[hi] and scale 0.1
    Fi = np.random.standard_cauchy() * 0.1 + M_F[hi]
    # Generate CRi using the Gaussian distribution with mean MCR[hi] and standard deviation 0.1
    CRi = np.random.normal(M_CR[hi], 0.1)
    # Ensure CRi is within the range [0, 1] and Fi is within the range [0,1]
    Fi = np.clip(Fi, 0, 1)
    CRi = np.clip(CRi, 0, 1)
    return Fi, CRi

# # Coba
# MF = [0.5, 0.6, 0.7, 0.8, 0.9] 
# MCR = [0.1, 0.2, 0.3, 0.4, 0.5]
# update_parameter(MF,MCR,len(MF))

In [11]:
def meanWL(elements, weights):
    """
    Calculate the weighted Lehmer mean of elements.
    Lehmer mean is calculated as the weighted sum of the squares
    divided by the weighted sum of the elements.
    """
    numerator = np.sum(np.multiply(np.square(elements), weights))
    denominator = np.sum(np.multiply(elements, weights))
    return numerator / denominator if denominator != 0 else 0

# Define the weighted arithmetic mean function
def meanWA(elements, weights):
    """
    Calculate the weighted arithmetic mean of elements.
    This is the standard weighted mean.
    """
    return np.average(elements, weights=weights)

def update_history(M_F,M_CR,S_F,S_CR,k):
    weights = np.array([1 for _ in range (len(S_F))])
    if len(S_F)!=0:
        M_F[k] = meanWL(S_F,weights) 
    if len(S_CR)!=0:
        M_CR[k] = meanWA(S_CR,weights)
    return M_F,M_CR


# RAGA Evaluation

In [12]:
import importlib.util

spec = importlib.util.spec_from_file_location("gal", r"D:\OneDrive - Institut Teknologi Bandung\[AKADEMIK]\Semester 7-8\TA\Thesis\Genetic Algorithm\genal.py")
gal = importlib.util.module_from_spec(spec)
spec.loader.exec_module(gal)

## Algorithm 2 - Gong

In [ ]:
# np.random.seed(seed)

# # Main RADE algorithm loop
# def RAGA(archive, 
#          objective_func,
#          bounds, 
#          population_size, 
#          max_generation, 
#          memories_F, 
#          memories_CR,
#          num_l,
#          theta,
#          tau_d,
#          archive_size_max,
#          print_gen = False):
    
#     dimensions = len(bounds)
#     population = generate_points(dim=dimensions,
#                                  npoint=population_size,
#                                  low=bounds[:,0],
#                                  high=bounds[:,1],
#                                  sobol=True)
#     fitness = np.asarray([objective_func(ind) for ind in population])
#     best_idx = np.argmin(fitness)
#     best = population[best_idx]
#     subpopulation = np.array([subpopulating(xi, population, num_l) for xi in population])
#     Hm = len(memories_F)
#     k=0

#     for gen in range(max_generation):
#         S_F, S_CR = [],[]
#         for i in range(population_size):
#             F_i,CR_i = update_parameter(memories_F,memories_CR,Hm)
#             selected_subpopulation
            
#             mutant = mutation_penalty(x_i,subpopulation[i],bounds,F_i)
#             trial = crossover(x_i, mutant, CR_i)
#             trial_fitness = fitness_function(trial, 
#                                              archive,
#                                              objective_func=objective_func,
#                                              repulsion_func=repulsion_function)
#             id_closest_trial,closest_trial = closest_solution(trial,population)
#             closest_trial_fitness = fitness_function(closest_trial, 
#                                                      archive,
#                                                      objective_func=objective_func,
#                                                      repulsion_func=repulsion_function)
#             if trial_fitness < closest_trial_fitness:
#                 # print(trial_fitness, fitness[i])
#                 fitness[id_closest_trial] = trial_fitness
#                 population[id_closest_trial] = trial
#                 archive = update_archive(x = trial,
#                                         objective_function=objective_func,
#                                         archive=archive,
#                                         theta=theta,
#                                         tau_d=tau_d,
#                                         s_max=archive_size_max)
#                 S_F.append(F_i)
#                 S_CR.append(CR_i)
#                 if trial_fitness < fitness[best_idx]:
#                     best_idx = i
#                     best = trial

#         if print_gen == True:
#             print(f"=========Generation {gen}=========")
#             # print(f"Best Fitness: {fitness[best_idx]}")
#             print(f'Archive:{archive}')
        
#             # print(f'S_F: {S_F}\nS_CR: {S_CR}')
#         """Update parameter history"""
#         if (len(S_F)!=0) & (len(S_CR)!=0):
#             memories_F,memories_CR = update_history(memories_F,memories_CR,S_F,S_CR,k)
#             # print(f'M_F: {M_F}\nM_CR: {M_CR}')
#             k +=1
#             if k >= Hm:
#                 k = 1
#     return best, fitness[best_idx], archive


# M_CR = np.ones(Hm)*CR_init
# M_F = np.ones(Hm)*F_init
# archiveA = []
# best_solution, best_fitness, archiveA = RAGA(objective_func=root_objective_function, 
#                                             archive=archiveA,
#                                             bounds = boundaries, 
#                                             population_size=pop_size, 
#                                             max_generation=max_gen, 
#                                             memories_F=M_F, 
#                                             memories_CR=M_CR,
#                                             num_l=num_l,
#                                             theta=theta,
#                                             tau_d=tau_d,
#                                             archive_size_max=s_max,
#                                             print_gen=True)
# print("Best Solution:", best_solution)
# print("Best Fitness:", best_fitness)
# print(f'Roots: {archiveA}')


In [13]:
M_CR = np.ones(Hm)*CR_init
M_F = np.ones(Hm)*F_init
archiveA = []

dimensions = len(boundaries)
population = generate_points(dim=dimensions,
                                npoint=pop_size,
                                low=boundaries[:,0],
                                high=boundaries[:,1],
                                sobol=True)
fitness = np.asarray([root_objective_function(ind) for ind in population])
best_idx = np.argmin(fitness)
best = population[best_idx]
subpopulation = np.array([subpopulating(xi, population, num_l) for xi in population])
Hm = len(M_F)
k=0

for gen in range(max_gen):
    S_F, S_CR = [],[]
    for i in range(0, pop_size, 2):
        F_i,CR_i = update_parameter(M_F,M_CR,Hm)
        x_i = population[i]
        selected_subpopulation1 = gal.selection(population=subpopulation[i],fitness=fitness)
        selected_subpopulation1 = sorted(selected_subpopulation1,key= lambda x: root_objective_function(x))
        selected_subpopulation2 = gal.selection(population=subpopulation[i+1],fitness=fitness)
        selected_subpopulation2 = sorted(selected_subpopulation2,key= lambda x: root_objective_function(x))
        parent1 = selected_subpopulation1[0]
        parent2 = selected_subpopulation2[0]
        offspring1, offspring2 = gal.crossover(parent1,parent2)
        offspring1 = mutate(offspring1,mutation_rate=F_i,boundaries=boundaries)
        offspring2 = mutate(offspring2,mutation_rate=F_i,boundaries=boundaries)        
        # __________________
        mutant = mutation_penalty(x_i,subpopulation[i],boundaries,F_i)
        trial = crossover(x_i, mutant, CR_i)
        trial_fitness = fitness_function(trial, 
                                            archive,
                                            objective_func=objective_func,
                                            repulsion_func=repulsion_function)
        id_closest_trial,closest_trial = closest_solution(trial,population)
        closest_trial_fitness = fitness_function(closest_trial, 
                                                    archive,
                                                    objective_func=objective_func,
                                                    repulsion_func=repulsion_function)
        if trial_fitness < closest_trial_fitness:
            # print(trial_fitness, fitness[i])
            fitness[id_closest_trial] = trial_fitness
            population[id_closest_trial] = trial
            archive = update_archive(x = trial,
                                    objective_function=objective_func,
                                    archive=archive,
                                    theta=theta,
                                    tau_d=tau_d,
                                    s_max=archive_size_max)
            S_F.append(F_i)
            S_CR.append(CR_i)
            if trial_fitness < fitness[best_idx]:
                best_idx = i
                best = trial

    if print_gen == True:
        print(f"=========Generation {gen}=========")
        # print(f"Best Fitness: {fitness[best_idx]}")
        print(f'Archive:{archive}')
    
        # print(f'S_F: {S_F}\nS_CR: {S_CR}')
    """Update parameter history"""
    if (len(S_F)!=0) & (len(S_CR)!=0):
        M_F,M_CR = update_history(M_F,M_CR,S_F,S_CR,k)
        # print(f'M_F: {M_F}\nM_CR: {M_CR}')
        k +=1
        if k >= Hm:
            k = 1
best, fitness[best_idx], archive


(100, 2) (100, 20, 2)
